In [ ]:
import h5py
from google.colab import files
import torch
import numpy as np
import time
from sklearn.manifold import TSNE
import plotly.express as px
from torch.utils.data import TensorDataset

In [ ]:
files.upload()

Saving vggish_features_labels.hdf5 to vggish_features_labels.hdf5


In [ ]:
with h5py.File('/content/vggish_features_labels.hdf5', 'r') as f:
  # file1 = f.get("data")
  data_file = np.array(f.get("data")).astype(float)

In [ ]:
print(data_file.shape)
print(type(data_file))
print(data_file[0][0])

(7077, 513)
<class 'numpy.ndarray'>
166.0


In [ ]:
features = data_file[:,:-1]
print("features", features.shape)
labels = data_file[:,-1]
print("labels", labels.shape)

features (7077, 512)
labels (7077,)


In [ ]:
dataset = TensorDataset( torch.Tensor(features), torch.Tensor(labels) )

In [ ]:
dataset

In [ ]:
loader = DataLoader(
        dataset,
        batch_size = 8,
        shuffle = False
  )

In [ ]:
features, labels = next(iter(loader))

In [ ]:
features.shape

torch.Size([8, 512])

In [ ]:
labels

tensor([2., 2., 2., 2., 2., 2., 2., 2.])

In [ ]:
start_time_tsne = time.time()

tsne = TSNE(n_components=2,random_state=42)
tsne_results = tsne.fit_transform(features)

fig = px.scatter(
        tsne_results, x=0, y=1,
        color=labels.astype(str),
        labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'}
      )
fig.show()

print('Total Time: %.2f min' % ((time.time() - start_time_tsne)/60))

Total Time: 1.04 min


In [ ]:
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00


In [ ]:
import import_ipynb
import os
from google.colab import files
from importlib import reload
if not os.path.exists("/content/Model_VAE.ipynb"): files.upload()
import Model_VAE as m_vae

if not os.path.exists("/content/Model_CVAE.ipynb"): files.upload()
import Model_CVAE as m_cvae

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import numpy as np

Saving Model_CVAE.ipynb to Model_CVAE.ipynb
Saving Model_VAE.ipynb to Model_VAE.ipynb
importing Jupyter notebook from Model_VAE.ipynb
importing Jupyter notebook from Model_VAE.ipynb
importing Jupyter notebook from Model_CVAE.ipynb
importing Jupyter notebook from Model_CVAE.ipynb


In [ ]:
import Model_VAE as m_vae
reload(m_vae)
import Model_CVAE as m_cvae
reload(m_cvae)

importing Jupyter notebook from Model_VAE.ipynb
importing Jupyter notebook from Model_CVAE.ipynb


<module 'Model_CVAE' from 'Model_CVAE.ipynb'>

In [ ]:
### SETTINGS

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

# Hyperparameters
random_seed = 0
learning_rate = 0.001
# learning_rate = 0.0001
num_epochs = 50
batch_size = 8

# Architecture
num_classes = 10
num_features = 512
# num_hidden_1 = 500
# num_hidden_1 = 512
num_hidden_1 = 512
# num_latent = 35
num_latent = 512

# train data
data = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = False # importante: para obtener resultados similares al re-ejecutar
      )

Device: cuda:0
VAE ready
CVAE ready


In [ ]:
# models
modelVAE = m_vae.VariationalAutoencoder(
              latent_dims= num_latent,
              num_features= num_features,
              num_hidden_layers= num_hidden_1
            )
modelVAE = modelVAE.to(device.type)
print("VAE ready")

modelCVAE = m_cvae.ConditionalVariationalAutoencoder(
              latent_dims= num_latent,
              num_features= num_features,
              num_hidden_layers= num_hidden_1,
              num_classes= num_classes
            )
modelCVAE = modelCVAE.to(device.type)
print("CVAE ready")

In [ ]:
print(type(modelVAE))
print(type(modelCVAE))
print(str(type(modelCVAE)))
if str(type(modelVAE)).__contains__('Model_CVAE'):
  print(1)
else:
  print(0)

<class 'Model_VAE.VariationalAutoencoder'>
<class 'Model_CVAE.ConditionalVariationalAutoencoder'>
<class 'Model_CVAE.ConditionalVariationalAutoencoder'>
0


In [ ]:
def to_onehot(labels, num_classes):
  # binariza las etiquetas, es decir convierte las etiquetas de clase en columnas y por fila, asigna un 1 en la columna que cumpla con la etiqueta
  labels_onehot = torch.zeros(labels.size()[0], num_classes).to(device.type)
  labels_onehot.scatter_(1, labels.view(-1, 1), 1)
  return labels_onehot

## training
def train(model, data, num_epochs=20, kl_version=2, reduction='sum'):
  start_time = time.time()

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  losses = []
  cvae = False
  model_name = str(type(model))
  if model_name.__contains__('Model_CVAE'):
    print('Training CVAE')
    cvae = True
  else:
    print('Training VAE')

  for epoch in range(num_epochs):
    loss_epoch = []
    for batch_idx, features_labels in enumerate(data):

      # print(features_labels[:,:-1].shape)
      # print(features_labels[:,-1].shape)
      # break

      features = features_labels[:,:-1].type(torch.float32).to(device) / 255.
      targets = features_labels[:,-1].type(torch.int32).to(device)

      # print(features[0])
      # print(targets)
      # break

      optimizer.zero_grad() #importante antes de usar modelo

      ### FORWARD
      if cvae:
        decoded = model(features, targets, kl_version)# Training CVAE

        # add condition
        x_con = torch.cat((features, to_onehot(targets, num_classes, device)), dim=1)

        ### BACK PROP
        # cost = reconstruction loss + Kullback-Leibler divergence

        kl_divergence = model.encoder.kl_divergence
        pixelwise_bce = torch.nn.functional.binary_cross_entropy(decoded, x_con, reduction=reduction)
        cost = kl_divergence + pixelwise_bce

      else:
        decoded = model(features) # Training VAE'
        # print("decoded", decoded)
        kl_divergence = model.encoder.kl_divergence
        cost = ((features - decoded)**2).mean() + kl_divergence

      ### UPDATE MODEL PARAMETERS
      cost.backward()
      optimizer.step()

      loss_epoch.append(cost)

      ### LOGGING
      if not batch_idx % num_epochs:
        print('%s | Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f | KL v%02d | reduction %s'
              %(model_name, epoch+1, num_epochs, batch_idx, len(data.dataset)/batch_size, cost, kl_version, reduction))

    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    losses.append((sum(loss)/len(loss)).item())

  print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))
  return model, losses


In [ ]:
(trained_vae, losses_vae) = train(modelVAE, data, kl_version=2, reduction='sum')

Training VAE
Epoch: 001/020 | Batch 000/000 | Cost: 0.5164
Epoch: 001/020 | Batch 020/000 | Cost: 0.1220
Epoch: 001/020 | Batch 040/000 | Cost: 0.0905
Time elapsed: 0.01 min
Epoch: 002/020 | Batch 000/000 | Cost: 0.0765
Epoch: 002/020 | Batch 020/000 | Cost: 0.0775
Epoch: 002/020 | Batch 040/000 | Cost: 0.0746
Time elapsed: 0.01 min
Epoch: 003/020 | Batch 000/000 | Cost: 0.0705
Epoch: 003/020 | Batch 020/000 | Cost: 0.0710
Epoch: 003/020 | Batch 040/000 | Cost: 0.0732
Time elapsed: 0.01 min
Epoch: 004/020 | Batch 000/000 | Cost: 0.0715
Epoch: 004/020 | Batch 020/000 | Cost: 0.0726
Epoch: 004/020 | Batch 040/000 | Cost: 0.0717
Time elapsed: 0.02 min
Epoch: 005/020 | Batch 000/000 | Cost: 0.0721
Epoch: 005/020 | Batch 020/000 | Cost: 0.0720
Epoch: 005/020 | Batch 040/000 | Cost: 0.0690
Time elapsed: 0.02 min
Epoch: 006/020 | Batch 000/000 | Cost: 0.0704
Epoch: 006/020 | Batch 020/000 | Cost: 0.0696
Epoch: 006/020 | Batch 040/000 | Cost: 0.0726
Time elapsed: 0.03 min
Epoch: 007/020 | Batc

In [ ]:
modelVAE.encoder.hidden_1.weight.dtype

torch.float32

In [ ]:
(trained_cvae, cost_cvae) = train(modelCVAE, data)

In [ ]:
from tqdm import tqdm

def getTrainedLatentVectorAndLabels(trained_model, data):
  start_time_pandas = time.time()

  encoded_samples = np.empty([0, num_latent])
  labels = np.empty([0,1])

  for features_labels in tqdm(data):
    # img = sample[0].squeeze(0).to(device.type)
    # features = sample[0].view(-1, num_features).to(device.type)
    # # label = sample[1].to(device.type)
    # targets = sample[1].to(device.type)

    features = features_labels[:,:-1].type(torch.float32).to(device) / 255.
    targets = features_labels[:,-1].type(torch.int32).to(device)

    trained_model.eval()
    with torch.no_grad():
      encoded_img = trained_model.encoder(features)

    labels = np.append(labels, targets.cpu().numpy())
    encoded_samples = np.append(encoded_samples, encoded_img.cpu().numpy(), axis=0)

  print('Total Time: %.2f min' % ((time.time() - start_time_pandas)/60))
  return encoded_samples, labels

In [ ]:
encoded_samples, labels = getTrainedLatentVectorAndLabels(trained_vae, data)

100%|██████████| 56/56 [00:00<00:00, 531.58it/s]

Total Time: 0.00 min


In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

start_time_tsne = time.time()

tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(encoded_samples)

fig = px.scatter(
        tsne_results, x=0, y=1,
        color=labels.astype(str),
        labels={'0': 'tsne-2d-one', '1': 'tsne-2d-two'}
      )
fig.show()

print('Total Time: %.2f min' % ((time.time() - start_time_tsne)/60))

Total Time: 1.18 min
